In [ ]:
#import libraries
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
import pandas as pd
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import numpy as np

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	""""Generate data for time series creating windows of given n."""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	# concat columns
	agg = concat(cols, axis=1)
	agg.columns = names
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [ ]:
def spacing_window(data,win_space):
  """"creating windows with specific frame."""
  data=data[data.index%win_space == 0]
  return data



In [ ]:
dataset_complete=pd.read_csv('/content/drive/MyDrive/data_final.csv')
x=max(dataset_complete['ID'])
y=min(dataset_complete['ID'])
print(x)
print(y)
TRAIN_X=np.array([])
TRAIN_Y=np.array([])
TEST_X=np.array([])
TEST_Y=np.array([])
for i in range (y,x+1):
  dataset=dataset_complete[dataset_complete['ID']== i]
  dataset1=dataset.filter(regex='_r$', axis=1)
  values = dataset1.values
  values = values.astype('float32')
  n_hours = 30
  n_features = 17
  # convert frame as time series data
  reframed = series_to_supervised(values, n_hours, 1)
  reframed=spacing_window(reframed,10)
  values = reframed.values
  n_train_hours = int( len(reframed) *0.9)
  train = values[:n_train_hours, :]
  test = values[n_train_hours:, :]
  n_obs = n_hours * n_features
  
  # test and train split
  train_X = train
  train_y = np.array([[dataset.iloc[0]['extraversion'],dataset.iloc[0]['neuroticism'],dataset.iloc[0]['agreeableness'],dataset.iloc[0]['conscientiousness'],dataset.iloc[0]['openness']]]*len(train))
  test_X = test
  test_y = np.array([[dataset.iloc[0]['extraversion'],dataset.iloc[0]['neuroticism'],dataset.iloc[0]['agreeableness'],dataset.iloc[0]['conscientiousness'],dataset.iloc[0]['openness']]]*len(test))
  
  train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
  test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))
  print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
  if len(TRAIN_X)==0:
    TRAIN_X=train_X
    TRAIN_Y=train_y
    TEST_X=test_X
    TEST_Y=test_y
  else:
    TRAIN_X=np.concatenate((TRAIN_X, train_X))
    TRAIN_Y=np.concatenate((TRAIN_Y, train_y))
    TEST_X=np.concatenate((TEST_X, test_X))
    TEST_Y=np.concatenate((TEST_Y, test_y))
  print('@@@@@@@@@@@@@@@@@@@@')
  print(TRAIN_X.shape, TRAIN_Y.shape, TEST_X.shape, TEST_Y.shape)



Streaming output truncated to the last 5000 lines.
(108056, 30, 17) (108056, 5) (13728, 30, 17) (13728, 5)
(38, 30, 17) (38, 5) (5, 30, 17) (5, 5)
@@@@@@@@@@@@@@@@@@@@
(108094, 30, 17) (108094, 5) (13733, 30, 17) (13733, 5)
(38, 30, 17) (38, 5) (5, 30, 17) (5, 5)
@@@@@@@@@@@@@@@@@@@@
(108132, 30, 17) (108132, 5) (13738, 30, 17) (13738, 5)
(38, 30, 17) (38, 5) (5, 30, 17) (5, 5)
@@@@@@@@@@@@@@@@@@@@
(108170, 30, 17) (108170, 5) (13743, 30, 17) (13743, 5)
(38, 30, 17) (38, 5) (5, 30, 17) (5, 5)
@@@@@@@@@@@@@@@@@@@@
(108208, 30, 17) (108208, 5) (13748, 30, 17) (13748, 5)
(38, 30, 17) (38, 5) (5, 30, 17) (5, 5)
@@@@@@@@@@@@@@@@@@@@
(108246, 30, 17) (108246, 5) (13753, 30, 17) (13753, 5)
(30, 30, 17) (30, 5) (4, 30, 17) (4, 5)
@@@@@@@@@@@@@@@@@@@@
(108276, 30, 17) (108276, 5) (13757, 30, 17) (13757, 5)
(32, 30, 17) (32, 5) (4, 30, 17) (4, 5)
@@@@@@@@@@@@@@@@@@@@
(108308, 30, 17) (108308, 5) (13761, 30, 17) (13761, 5)
(40, 30, 17) (40, 5) (5, 30, 17) (5, 5)
@@@@@@@@@@@@@@@@@@@@
(108348, 30, 

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras import layers
from keras import Model
from tensorflow.keras import regularizers
# model architecture and training
inputs = layers.Input(shape=(n_timesteps,n_features))
input=layers.Conv1D(filters=256, kernel_size=3, activation='relu')(inputs)
hid1=layers.Conv1D(filters=256, kernel_size=3, activation='relu')(input)
reg = layers.MaxPooling1D(pool_size=2)(hid1)
reg2 = layers.MaxPooling1D(pool_size=2)(reg)
flat=layers.Flatten()(reg2)
hid3=layers.Dense(100, activation='relu')(flat)
hid2=layers.Dense(100, activation='relu')(hid3)
out1=Dense(1, activation='sigmoid')(hid2)
out2=Dense(1, activation='sigmoid')(hid2)
out3=Dense(1, activation='sigmoid')(hid2)
out4=Dense(1, activation='sigmoid')(hid2)
out5=Dense(1, activation='sigmoid')(hid2)
model = Model(inputs=inputs, outputs=[out1, out2,out3,out4,out5])
model.compile(optimizer='adam', loss=['binary_crossentropy','binary_crossentropy','binary_crossentropy','binary_crossentropy','binary_crossentropy'],metrics='accuracy')
history = model.fit(TRAIN_X,[TRAIN_Y[:,0],TRAIN_Y[:,1],TRAIN_Y[:,2],TRAIN_Y[:,3],TRAIN_Y[:,4]], epochs=50)

Epoch 1/50
7181/7181 [==============================] - 399s 55ms/step - loss: 2.8285 - dense_9_loss: 0.5689 - dense_10_loss: 0.5852 - dense_11_loss: 0.5709 - dense_12_loss: 0.5938 - dense_13_loss: 0.5096 - dense_9_accuracy: 0.6965 - dense_10_accuracy: 0.6793 - dense_11_accuracy: 0.7045 - dense_12_accuracy: 0.6721 - dense_13_accuracy: 0.7476
Epoch 2/50
7181/7181 [==============================] - 393s 55ms/step - loss: 2.4649 - dense_9_loss: 0.4957 - dense_10_loss: 0.5093 - dense_11_loss: 0.5017 - dense_12_loss: 0.5134 - dense_13_loss: 0.4449 - dense_9_accuracy: 0.7435 - dense_10_accuracy: 0.7368 - dense_11_accuracy: 0.7496 - dense_12_accuracy: 0.7331 - dense_13_accuracy: 0.7890
Epoch 3/50
7181/7181 [==============================] - 358s 50ms/step - loss: 2.2483 - dense_9_loss: 0.4553 - dense_10_loss: 0.4627 - dense_11_loss: 0.4598 - dense_12_loss: 0.4670 - dense_13_loss: 0.4036 - dense_9_accuracy: 0.7700 - dense_10_accuracy: 0.7690 - dense_11_accuracy: 0.7743 - dense_12_accuracy: 0.7

In [ ]:
model.save('AU_cnn_diversifed_256x')

INFO:tensorflow:Assets written to: AU_cnn_diversifed_256x/assets


In [ ]:

!cp -r AU_cnn_diversifed_256x  /content/drive/MyDrive

In [ ]:
prediction=model.predict(TEST_X)
score = model.evaluate(TEST_X, [TEST_Y[:,0],TEST_Y[:,1],TEST_Y[:,2],TEST_Y[:,3],TEST_Y[:,4]], verbose = 0) 

In [ ]:
print(prediction[2].shape)

(29164, 1)


In [ ]:
print(score)
count=0
for i in range(0,29164):
  
  pred=prediction[4][i]
  k_pred=round(pred[0],0)
  k=[TEST_Y[:,0],TEST_Y[:,1],TEST_Y[:,2],TEST_Y[:,3],TEST_Y[:,4]]
  k_act=k[4][i]
  if k_act==k_pred:
    count=count+1
print(count/29164)

[5.444511413574219, 1.0852993726730347, 1.175935983657837, 1.1197724342346191, 1.0718218088150024, 0.9916802644729614, 0.7538403272628784, 0.7462282180786133, 0.7472912073135376, 0.740261971950531, 0.7865176200866699]
0.7865176244685228


In [ ]:
from tensorflow import keras
model = keras.models.load_model('/content/drive/MyDrive/AU_lstm_diversifed')

In [ ]:
prediction=model.predict(TEST_X)
print(prediction[2].shape)

(83085, 1)
